# Nombre: Claudio Maldonado


In [1]:
# install
!pip install plotly
!pip install datapane

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 636 kB 42.0 MB/s 
     |████████████████████████████████| 193 kB 67.0 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 10.1 MB 44.5 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
     |████████████████████████████████| 46 kB 2.9 MB/s 
  Created wheel for face: filename=face-20.1.1-py3-none-any.whl size=51091 sha256=71fdc2de40bb562dd107aef312edb2c2af36349679e15e99a7b2971f01995a6d
  Stored in directory: /root/.cache/pip/wheels/67/03/c2/76d2e1e694852ce409ac2938b9f461bc0fdf3789901e7bb1f0
  Created wheel for micawber: filename=micawber-0.5.4-py3-none-any.whl size=22373 sha256=e157c18b58e1e71455cfa70e6004da37cc13419f09be8374e88186c7ce5b4152
  Stored in directory: /

In [2]:
# Imports
import pandas as pd
import numpy as np
import plotly as ply
import plotly.graph_objects as go
import plotly.express as px
import datapane as dp

In [3]:
df = pd.read_csv("/content/1.Poblaciones.csv", encoding='latin1', sep = ';', skiprows=2)

df.rename(columns ={
    'Unnamed: 0':'Encuesta','Unnamed: 1':'Periodo','Unnamed: 2':'Indicadores', 'Unnamed: 5':'Area'
    ,'Unnamed: 7':'Sexo'
}, inplace = True)
df = df.drop(df.columns[[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]], axis=1) 
df["Total"] = df["Total"].str.replace(".", "").astype(int)
df["Urbana"] = df["Urbana"].str.replace(".", "").astype(int)
df["Rural"] = df["Rural"].str.replace(".", "").astype(int)
df["Hombre"] = df["Hombre"].str.replace(".", "").astype(int)
df["Mujer"] = df["Mujer"].str.replace(".", "").astype(int)

df.head(10)

,Encuesta,Periodo,Indicadores,Total,Urbana,Rural,Hombre,Mujer
0,ENEMDU,dic-07,Población Total,13682302,9066209,4616093,6768646,6913656
1,ENEMDU,dic-07,Población menor de 15 años,4372812,2723124,1649688,2226618,2146194
2,ENEMDU,dic-07,Población en Edad de Trabajar (PET),9309490,6343085,2966404,4542028,4767462
3,ENEMDU,dic-07,Población Económicamente Activa,6336029,4227702,2108328,3777232,2558798
4,ENEMDU,dic-07,Empleo,6019332,3971040,2048292,3632314,2387018
5,ENEMDU,dic-07,Empleo Adecuado/Pleno,2737158,2236440,500719,1907451,829707
6,ENEMDU,dic-07,Subempleo,1155872,687714,468158,679118,476753
7,ENEMDU,dic-07,Subempleo por insuficiencia de tiempo de trabajo,860315,519627,340688,474355,385960
8,ENEMDU,dic-07,Subempleo por insuficiencia de ingresos,295556,168087,127470,204763,90793
9,ENEMDU,dic-07,Empleo no remunerado,557146,215994,341152,176122,381025


In [4]:
# Empleo y Desempleo
def empleo_desempleo(df_period):
  df_filter_empleo = df_period.loc[(df_period['Indicadores'] == 'Empleo')]
  df_filter_desempleo = df_period.loc[(df_period['Indicadores'] == 'Desempleo')]
  df_by_periodo_empleo = df_filter_empleo.groupby(['Periodo'], as_index=False)['Total'].mean()
  df_by_periodo_empleo.rename(columns ={
    'Total':'Empleo'
  }, inplace = True)
  df_by_periodo_desempleo = df_filter_desempleo.groupby(['Periodo'], as_index=False)['Total'].mean() 
  df_by_periodo_desempleo.rename(columns ={
    'Total':'Desempleo'
  }, inplace = True)
  df_ed_res = pd.merge(df_by_periodo_empleo,df_by_periodo_desempleo, on='Periodo')
  x_data=df_ed_res['Periodo']
  ed_fig = go.Figure(data=[
    go.Bar(name='Empleo', x=x_data, y=df_ed_res['Empleo']),
    go.Bar(name='Desempleo', x=x_data, y=df_ed_res['Desempleo'])
  ])
  ed_fig.update_yaxes(title="Total")
  ed_fig.update_xaxes(title="Año")
  ed_fig.update_layout(barmode='group')
  ed_fig.show()
  return ed_fig, df_ed_res


In [5]:
# Sectorización de empleo
def empleo_sector(df_period):
  df_es_res = df_period.groupby(['Periodo', 'Indicadores'], as_index=False)['Total'].mean()
  df_es_res.rename(columns ={
    'Indicadores':'Sector'
  }, inplace = True)
  es_fig = px.pie(df_es_res, values='Total', names='Sector', title='Sectorización de Empleo')
  es_fig.show()
  return es_fig, df_es_res

In [6]:
# Subempleo, Empleo pleno y Empleo no pleno
def empleo_sub_pleno(df_period):
  df_filter_sub_empleo = df_period.loc[(df_period['Indicadores'] == 'Subempleo')].groupby(['Periodo', 'Indicadores'], as_index=False)['Total'].mean()
  df_filter_empleo_pleno = df_period.loc[(df_period['Indicadores'] == 'Empleo Adecuado/Pleno')].groupby(['Periodo', 'Indicadores'], as_index=False)['Total'].mean()
  df_filter_empleo_no_pleno = df_period.loc[(df_period['Indicadores'] == 'Otro Empleo no pleno')].groupby(['Periodo', 'Indicadores'], as_index=False)['Total'].mean()

  x_data=df_filter_sub_empleo['Periodo']
  esp_fig = go.Figure(data=[
    go.Bar(name='Subempleo', x=x_data, y=df_filter_sub_empleo['Total']),
    go.Bar(name='Empleo Pleno', x=x_data, y=df_filter_empleo_pleno['Total']),
    go.Bar(name='Empleo no pleno', x=x_data, y=df_filter_empleo_no_pleno['Total'])
  ])
  esp_fig.update_yaxes(title="Total")
  esp_fig.update_xaxes(title="Año")
  esp_fig.update_layout(barmode='group')
  esp_fig.show()
  return esp_fig, df_filter_sub_empleo, df_filter_empleo_pleno, df_filter_empleo_no_pleno

In [7]:
def data_report(init='07', end='21'):
  try:
    df_period = df.copy()
    df_period['Periodo'] = df_period['Periodo'].str[-2:]
    df_period = df_period.loc[(df_period['Periodo']>=init) & (df_period['Periodo']<=end)]
    # Empleo y Desempleo
    ed_fig, df_ed_res = empleo_desempleo(df_period)
    # Sectorización de empleo
    es_fig, df_es_res = empleo_sector(df_period)
    # Subempleo, Empleo pleno y Empleo no pleno
    esp_fig, df_filter_sub_empleo, df_filter_empleo_pleno, df_filter_empleo_no_pleno = empleo_sub_pleno(df_period)
    report = dp.Report(dp.Text("""## Descripción de datos"""), dp.Text("""## Descripción de datos"""), dp.Table(df.describe()), dp.Text("""## Empleo y Desempleo"""), dp.Plot(ed_fig),dp.Text("""## Sectorización de Empleo"""), dp.Plot(es_fig),dp.Text("""## Sectorización de Subempleo, Empleo Pleno, Empleo no pleno"""), dp.Plot(esp_fig), dp.Text("""## Conclusión"""),dp.Text("""Para este trabajo se realizó una limpieza de los datos con el fin de obtener mejores resultados, para generar el reporte se usa graficas dinámicas la cual facilita la visualización de los datos y con la librería de plotly esto se hace un trabajo sencillo."""), dp.Text("""Los reportes se generan en base a los años enviados por parámetros esto ayuda a segmentar los datos para una mejor visualización."""))
    #genera el reporte en html
    file_name = "report-"+init+end+".html"
    report.save(path=file_name,open=True)
  except:
    print("Error rango de años invalido rangos permitidos desde 07 hasta el 21")

In [8]:
init = input('Ingrese el año de inicio: ')
end = input('Ingrese el año de fin: ')

data_report(init[-2:], end[-2:])

Ingrese el año de inicio: 07
Ingrese el año de fin: 21


Bokeh version 2.3.3 is not supported, these plots may not display correctly, please install version ~=2.2.0
Folium version 0.8.3 is not supported, these plots may not display correctly, please install version >=0.12.0


Report saved to ./report-0721.html. To upload and share your report, create a free Datapane account by running `datapane signup`.
How is your experience of Datapane? Please take two minutes to answer our anonymous product survey at https://bit.ly/3lWjRlr


## Conclusión



*   Para este trabajo se realizó una limpieza de los datos con el fin de obtener mejores resultados, para generar el reporte se usa graficas dinámicas la cual facilita la visualización de los datos y con la librería de plotly esto se hace un trabajo sencillo.
*   Los reportes se generan en base a los años enviados por parámetros esto ayuda a segmentar los datos para una mejor visualización.
* En la grafica el indice de personas decempleadas completamente es de miles con respecto a las personas empleadas que supera los millones, en el caso de sub empleo, empleo pleno, empleo no pleno el indice de subempleo es superior a estos 2.


## Recomendaciones
La recomendación en base a este trabajo es limpiar los datos para evitar errores en la descripción de los datos, y mantener un correlación con lo que se quiere obtener.